### Homework 1 
    8-puzzle & 15-puzzle  
    Gianluca Rea

### Libraries and variables definition

In [1]:
### Libraries import
import numpy as np
import random
import time
from copy import deepcopy

In [2]:
### On the board an empty cell is represented using the value 0
step = ["up","down","left","right"];

### Functions and Heuristics

In [3]:
### The following two function are realized to prevent the creation of non solvible board 
def getInvCount(arr,n):
    invCount = 0
    emptyValue = 0
    for i in range (0,n*n):
        for j in range (i+1,n*n):
            if arr[j] != emptyValue and arr[i] != emptyValue and arr[i] > arr[j]:
                invCount += 1
    return invCount

def hasSolution(puzzle,n):
    invCount = getInvCount([j for sub in puzzle for j in sub], n)
    if(n%2==1): return (invCount % 2 == 0)    
    blankPos = 4 - np.where(puzzle == 0)[0]
    return (invCount % 2 != blankPos % 2)


In [4]:
### createWorld(N) create a board of dimension n using the previous 
### function to create a solvible one.

def createWorld(n):
    board =[i for i in range(1,n*n)]
    board.append(0)
    board = np.array(board).reshape(n,n)
    while(True):
        s0 = np.arange(n*n)
        np.random.shuffle(s0)
        s0 = s0.reshape(n,n)
        if(hasSolution(s0, n)): break
    return s0, board

s0, board = (createWorld(4))

In [5]:
### The function neighbors(state,n) given the current state and dimension n of the building 
### array of the matrix n gives back the position of the empty cell and the all the possible 
### steps even the ones not feasible 

def neighbors(state, n):
    i, j = np.where(state == 0)
    i, j = i[0], j[0]
    up =  (i-1,j) if (i-1>-1) else (None)
    down =  (i+1,j) if (i+1<n) else (None)
    right =  (i,j+1) if (j+1<n) else (None)
    left =  (i,j-1) if (j-1>-1) else (None)
    
    return (i,j),[up, down, left, right]

for indx, val in enumerate(neighbors(s0, 4)[1]):
    
    if val != None:
        i, j = val
        print(step[indx],":", i, j)
    else :
        print(step[indx],":", None)
 

up : 0 1
down : 2 1
left : 1 0
right : 1 2


In [6]:
### Manahattan distance heuristic

def manahattanDistance(state, n):
    mDistances = 0
    for index in range(1,n*n):
        iGoal, jGoal = np.where(board == index)
        iValue, jValue = np.where(state == index)
        manahattanDistanceVar = abs(iValue - iGoal) + abs(jValue - jGoal)
        mDistances +=  manahattanDistanceVar 
    return mDistances[0]


In [7]:
### Misplaced tiles heuristic
### We use this heuristics to count how many pieces are misplaced 

def misplacedTiles(state, n):
    misplacedTilesVar = 0
    for indx, val in enumerate(state.reshape(n*n)):
        if indx +1 != val and val != 0 :
            misplacedTilesVar = misplacedTilesVar +1
    
    return misplacedTilesVar

In [8]:
### The function makeMoves takes as input the state, the position of the empy cell and all the feasible moves
### and as output a list with all the new states

def makeMoves(boardState, positionZero , feasibleMoves ):
    newStatesList = []
    iZero, jZero = positionZero
    for move in feasibleMoves:
        if move != None:
            i, j = move
            newState = np.copy(boardState)
            newState[iZero,jZero], newState[i,j] = newState[i,j], newState[iZero,jZero]
            newStatesList.append(newState)
            
    return newStatesList

### Agents

In [9]:
### All the agents are optimised , for each new state created from the previous one
### we check if the new state are equal to the current one. This way we don't explore every tree 
### saving memory and execution time

### Agent 1 implements the Manahattan Distance heuristic
### Agent 2 implements the Misplaced Tiles heuristic

In [10]:
def agent1(s0, n, board):
    startTime = time.time()

    currentState = s0
    previousState = s0
    i = 0
    while (not np.allclose(board, currentState)):
        i+=1   
        positionZero, feasibleMoves = (neighbors(currentState, n ))                             
        newStateBoards = makeMoves(currentState, positionZero , feasibleMoves)    
        heuristicValues = []
        boardStates = []
        for newStateBoard in newStateBoards:
            if not np.allclose(newStateBoard, previousState):
                boardStates.append(newStateBoard)
                heuristicValues.append(manahattanDistance(newStateBoard, n))
        indexMin = random.choice([i for i, x in enumerate(heuristicValues) if x == min(heuristicValues)])
        previousState = currentState
        currentState = boardStates[indexMin]
    print("--- %s seconds ---" % (time.time() - startTime))

    return(currentState,i)

In [11]:
def agent2(s0, n, board):
    startTime = time.time()

    currentState = s0
    previousState = s0
    i = 0
    while (not np.allclose(board, currentState)):
        i+=1   
        positionZero, feasibleMoves = (neighbors(currentState, n ))                            
        newStateBoards = makeMoves(currentState, positionZero, feasibleMoves)    
        heuristicValues = []
        boardStates = []
        for newStateBoard in newStateBoards:
            if not np.allclose(newStateBoard, previousState):
                boardStates.append(newStateBoard)
                heuristicValues.append(misplacedTiles(newStateBoard, n))
        indexMin = random.choice([i for i, x in enumerate(heuristicValues) if x == min(heuristicValues)])
        previousState = currentState
        currentState = boardStates[indexMin]
    print("--- %s seconds ---" % (time.time() - startTime))

   
    return(currentState,i)

### Bredth First Search and Depth First Search Implementation

In [12]:
### The class node is needed for modeling the tree,
### in this way we can easily implement a DFS and BFS

class Node(object):
    def __init__(self, name: int, data: np.array, parent, notVisited = None):
        self.name = name
        self.data = data
        self.parent = parent
        self.notVisited = notVisited

    def getParent(self) :
        return self.parent

In [13]:
### BFS
def bredthFirstSearch(puzzle: np.array, n, board) : 
    startTime = time.time()

    sol = False
    firstNode = Node(1,puzzle, board)
    explorationNodes = [firstNode]
    j = 0
    i = 1
    while(not sol):
        j = j+1
        listX = []
        print(j, len(explorationNodes))
        for node in explorationNodes:
            if(np.allclose(node.data, board)):
                sol = True
                print("--- %s seconds ---" % (time.time() - startTime))
                return node.data
               
            else:
               
                index, feasibleMoves = neighbors(node.data, n)
                toExplore =  makeMoves(node.data, index, feasibleMoves)
               
                for nodeX in toExplore:
                    i = i+1
                    if (not np.allclose(nodeX, node.getParent().data )):
                        child = Node(i+1, nodeX, node)
                        listX.append(child)
               
            explorationNodes = listX

In [14]:
def depthFirstSearch(puzzle: np.array, n, board) : 
    startTime = time.time()
    stack = [] 
    visited = set()
    gol = Node(-1, board,None, [])
    firstNode = Node(0, puzzle,gol, [])
    stack.append(deepcopy(firstNode)) 
    while (len(stack) !=0): 
        newNode = stack.pop() 
        if newNode not in visited:
            if (np.allclose(newNode.data, board)):
                print("--- %s seconds ---" % (time.time() - startTime))
                return (newNode.name, newNode.data)
            visited.add(newNode) 
        index, feasibleMoves = neighbors(newNode.data, n)
        newNode.notVisited =  makeMoves(newNode.data, index, feasibleMoves)
        newNode.notVisited.reverse()
        for neighbor in newNode.notVisited:
            if not np.allclose(neighbor, newNode.parent.data):
                neighborNode = Node(i, neighbor, newNode, [])
                if neighborNode not in visited: 
                       stack.append(neighborNode) 

# Tests

In [17]:
#Easy example
n = 3
Q, board = (createWorld(n))
print()
print(agent1(Q, n, board))
print()
print(agent2(Q, n, board))


--- 0.05060696601867676 seconds ---
(array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 0]]), 110)

--- 0.41634511947631836 seconds ---
(array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 0]]), 4044)


In [18]:
### We show number of iteration, cardinality of explored nodes, execution time
### and solution
print(bredthFirstSearch(Q, n, board))

1 1
2 4
3 8
4 8
5 16
6 32
7 64
8 80
9 160
10 272
11 544
12 752
13 1504
14 2384
15 4768
16 6896
17 13792
18 21200
19 42400
20 62576
21 125152
22 189776
23 379552
--- 53.93747282028198 seconds ---
[[1 2 3]
 [4 5 6]
 [7 8 0]]


In [ ]:
# Only in easy solvible board is possible to find a solution using DFS
print(depthFirstSearch(Q, n, board))


In [ ]:
n = 4
Q, board = (createWorld(n))
print()
print(agent1(Q, n, board))
print()
print(agent2(Q, n, board))
